# 导入模块

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
import numpy as np
%matplotlib inline

# 划分测试集和训练集

In [ ]:
import pandas as pd
from sklearn.model_selection  import train_test_split
csv_file = "data.csv"
df = pd.read_csv(csv_file,header=None)
df.tail(10)
train, test = train_test_split(df, test_size=0.333,random_state=233)
train.to_csv("train.csv",header=False,index=False)
test.to_csv("test.csv",header=False,index=False)

# 重写数据读入类

In [2]:
class ToTensor(object):
    def __call__(self, feature, label):
        return torch.from_numpy(feature), label-1

class Compose(object):

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, feature, label):
        for t in self.transforms:
            feature, label = t(feature, label)
        return feature, label
class ACTDataset(torch.utils.data.Dataset):
    def __init__(self,csvfile,transform=None):
        self.dataframe = pd.read_csv(csvfile)
        self.transform=transform
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        label = self.dataframe.iloc[idx,-1]
        feature = self.dataframe.iloc[idx,:-1]
        feature = np.asarray(feature,dtype=np.float32)
        if self.transform:
            feature, label = self.transform(feature,label)
        return label, feature


# 训练集和测试集定义

In [3]:
transform = Compose([ToTensor(),])
train_file="train.csv"
test_file="test.csv"
trainset = ACTDataset(train_file,transform)
testset  = ACTDataset(test_file,transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)
dataiter = iter(trainloader)
labels, feature = dataiter.next()
print(feature.shape)

torch.Size([4, 11092])


# MLP 定义

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(11092, 4096)
        self.fc2 = nn.Linear(4096, 512)
        self.fc3 = nn.Linear(512, 5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x))
net = MLP()
print(net)

MLP(
  (fc1): Linear(in_features=11092, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=5, bias=True)
)


# 定义损失函数和优化方法



In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 训练网络


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
net=net.to(device)
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        labels, inputs = data
        labels, inputs = labels.to(device), inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

cuda:0


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


[1,    20] loss: 1.606
[1,    40] loss: 1.598
[1,    60] loss: 1.587
[1,    80] loss: 1.574
[1,   100] loss: 1.560
[1,   120] loss: 1.556
[1,   140] loss: 1.533
[1,   160] loss: 1.497
[1,   180] loss: 1.477
[1,   200] loss: 1.429
[1,   220] loss: 1.371
[1,   240] loss: 1.429
[1,   260] loss: 1.381
[1,   280] loss: 1.350
[2,    20] loss: 1.304
[2,    40] loss: 1.329
[2,    60] loss: 1.311
[2,    80] loss: 1.269
[2,   100] loss: 1.292
[2,   120] loss: 1.226
[2,   140] loss: 1.281
[2,   160] loss: 1.271
[2,   180] loss: 1.236
[2,   200] loss: 1.213
[2,   220] loss: 1.188
[2,   240] loss: 1.165
[2,   260] loss: 1.181
[2,   280] loss: 1.179
[3,    20] loss: 1.075
[3,    40] loss: 1.057
[3,    60] loss: 1.065
[3,    80] loss: 1.036
[3,   100] loss: 1.027
[3,   120] loss: 1.025
[3,   140] loss: 1.022
[3,   160] loss: 1.024
[3,   180] loss: 1.004
[3,   200] loss: 0.971
[3,   220] loss: 1.056
[3,   240] loss: 1.024
[3,   260] loss: 0.990
[3,   280] loss: 0.968
[4,    20] loss: 0.951
[4,    40] 

[25,   240] loss: 0.905
[25,   260] loss: 0.905
[25,   280] loss: 0.905
Finished Training


# 检验准确率


In [7]:
correct = 0
total = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
truth=[]
guess=[]
with torch.no_grad():
    for data in testloader:
        labels, inputs = data
        labels, inputs = labels.to(device), inputs.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        truth.append(predicted)
        guess.append(labels)
        correct += (predicted == labels).sum().item()
a=torch.cat(truth,0)
b=torch.cat(guess,0)
print(a.shape)
print(len(testset),len(trainset))
c=torch.stack((a,b))
c=c.t()
c=c.cpu()
c=c.numpy()
rdf=pd.DataFrame(c)
rdf.to_csv("result.csv",header=False,index=False)
print(rdf.head(10))
print('Accuracy of the network : %d %%' % (
    100 * correct / total))

cuda:0


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


torch.Size([574])
574 1150
   0  1
0  3  3
1  2  4
2  3  3
3  0  2
4  0  0
5  4  4
6  1  1
7  0  0
8  2  2
9  2  2
Accuracy of the network : 83 %


In [8]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
with torch.no_grad():
    for data in testloader:
        labels, inputs = data
        labels, inputs = labels.to(device), inputs.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

classes=('sit','stand','jump','lie','walk')
for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Accuracy of   sit : 95 %
Accuracy of stand : 83 %
Accuracy of  jump : 55 %
Accuracy of   lie : 98 %
Accuracy of  walk : 85 %
